In [1]:
import pandas as pd
import numpy as np
from prize import Idx, Product, Basket
import logging
# change logging level from INFO to DEBUG to print debugging logs
logging.basicConfig(level=logging.INFO, 
                    format='%(levelname)s - %(funcName)s - %(lineno)d - %(message)s')


# Configurations:
file_name = "./products.csv"

# specify the number of rows to read, set to "None" to read the whole file
num_rows = None 

# The tote’s usable space is 45 centimeters long, 30 wide and 35 high
tote_volume = 45 * 30 * 35  # 47250 cm3

In [2]:
# read input file

# Format of inputs
#  |     0      |   1   |    2   |   3   |   4    |   5    |
#  | product ID | price | length | width | height | weight |
#  |     /      | cents |   cm   |  cm   |   cm   |   g    |
# -------------------------------------------------------------
inputs = pd.read_csv(file_name, header=None, 
                     names=list(range(6)), nrows=num_rows)  # DataFrame
row, column = inputs.shape
logging.info("'{}' - row = {}, column = {}".format(file_name, row, column))
# print(inputs)

INFO - <module> - 11 - './products.csv' - row = 20000, column = 6


In [3]:
lengths = inputs.iloc[:, Idx.length].values  # Numpy Array
widths = inputs.iloc[:, Idx.width].values    # Numpy Array
heights = inputs.iloc[:, Idx.height].values  # Numpy Array

# The tote’s usable space is 45 centimeters long, 30 wide and 35 high
print(len(lengths[lengths > 45]))
print(len(widths[widths > 45]))
print(len(heights[heights > 45]))


1165
53
0


In [4]:
volumes = []    # List
densities = []  # List
for i in range(row):
    volumes.append(inputs.iloc[i, Idx.length] * inputs.iloc[i, Idx.width] * inputs.iloc[i, Idx.height])
    densities.append(volumes[i] / inputs.iloc[i, Idx.price])

min_volume = min(volumes)

print("minimum volume of a product = {} cm3".format(min_volume))
print("maximum no. of products can fit into the tote = {}".format(tote_volume//min_volume))

inputs[Idx.volume] = pd.Series(volumes, index=inputs.index)
inputs[Idx.density] = pd.Series(densities, index=inputs.index)

# Format of inputs
#  |     0      |   1   |    2   |   3   |   4    |   5    |    6   |    7     |
#  | product ID | price | length | width | height | weight | volume | density  |
#  |     /      | cents |   cm   |  cm   |   cm   |   g    |   cm3  | cent/cm3 |
# ------------------------------------------------------------------------------


# The tote’s usable space is 45 centimeters long, 30 wide and 35 high
# Assume that the orientation of the product doesn't matter 
# (i.e. no request to place the product upright always).
# For simplicity, only consider the potential rotation of the product be 90, 180, 270 degrees.
c_30, c_35, c_45, c_candidate = 0, 0, 0, 0
for i in range(row):
    dimensions = sorted([inputs.iloc[i, Idx.length], inputs.iloc[i, Idx.width], 
                         inputs.iloc[i, Idx.height]])  # ascending order
    if dimensions[0] > 30:
        c_30 += 1
    if dimensions[1] > 35:
        c_35 += 1
    if dimensions[2] > 45:
        c_45 += 1
    if dimensions[0] > 30 or dimensions[1] > 35 or dimensions[2] > 45:
        # the product doesn't fit into the tote
        continue
    c_candidate += 1

print("no. of products whose min dimension > 30cm = {}".format(c_30))
print("no. of products whose middle dimension > 35cm = {}".format(c_35))
print("no. of products whose max dimension > 45cm = {}".format(c_45))
print("no. of products bigger than the tote = {}".
      format(sum([item > tote_volume for item in volumes])))
print("no. of candidate products = {}".format(c_candidate))

print("inputs.shape = {}".format(inputs.shape))

minimum volume of a product = 1500 cm3
maximum no. of products can fit into the tote = 31


no. of products whose min dimension > 30cm = 356
no. of products whose middle dimension > 35cm = 1101
no. of products whose max dimension > 45cm = 1165
no. of products bigger than the tote = 366
no. of candidate products = 17933
inputs.shape = (20000, 8)


In [ ]:
# total no. of candidate products = 17933 (out of 20000)
